In [1]:
import torch
from torch import nn
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob
import datetime
import random

In [2]:
#GET FAT AND WEIGHT DATA

#things to fill out
rootdir = "maddy driscoll/MyFitbitData5/Mad/personal & Account"
regex = re.compile('weight-2021-03-05')
startDate = '03/13/21' 
endDate = '03/28/21'

#get time period
startd = datetime.datetime.strptime(startDate, "%m/%d/%y")
endd = datetime.datetime.strptime(endDate, "%m/%d/%y")#

#gets range of dates between start and end
date_array = (startd + datetime.timedelta(days=x) for x in range(0, (endd-startd).days))
dates = []
for date_object in date_array:
    #gets date value as string
    dates = np.append(dates, date_object.strftime("%m/%d/%y"))

    
#get weight files
filenames = os.listdir(rootdir)
weights= []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            weights = np.append(weights, file)
print(weights)            


#get weight data
roughdata = []
for x in range(np.size(weights)):
    filename = rootdir + "/" + weights[x]    
    data = []
    weight = []
    fat = []
    with open(filename) as json_file:
        rawdata = json.load(json_file)
        
        #checks if data is within start and end date
        for n in range(np.size(rawdata)):
            date = rawdata[n]["date"]
            if date in dates:
                roughdata.append(rawdata[n])  
                
        #delete irrelevant information then save relevant info
        for x in range(np.size(roughdata)):    
            del roughdata[x]["logId"]
            del roughdata[x]["time"]
            del roughdata[x]["source"]
            del roughdata[x]["bmi"]
            weight = np.append(weight, roughdata[x]["weight"])
            fat = np.append(fat, roughdata[x]["fat"])

        data = np.append(data, roughdata)

print(fat)
print(weight)

['weight-2021-03-05.json']
[15.30000019 14.6        13.89999962 13.19999981 13.19999981 14.60000038
 14.60000038 14.60000038 13.89999962 13.89999962 13.89999962 14.60000038
 14.60000038 13.19999981 13.19999981]
[114.6 114.6 112.  112.8 112.8 112.  112.2 110.  110.8 109.8 109.8 110.4
 107.6 109.8 108.7]


In [3]:
#GET CALORIE DATA

#things to fill out
rootdir = "maddy driscoll/MyFitbitData5/Mad/Nutrition"
regex = re.compile('food_logs-0')
startD = "2021-03-12"
endD = "2021-03-27"

#get calorie files
filenames = os.listdir(rootdir)
foodfiles = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            foodfiles = np.append(foodfiles, file)


#start boolean for when data is within date range
start = False

#get food entries
for x in range(np.size(foodfiles)):
    filename = rootdir + "/" + foodfiles[x]
    data = []
    count = 0
    
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        for y in range(np.size(roughdata)):
            if roughdata[y-count]["logDate"] == startD:
                start = False
            if roughdata[y-count]["logDate"] == endD:
                start = True
                
            #once within date range:
            if start == True:
                #delete irrelevant informaiton and track calorie information
                cal = roughdata[y-count]["loggedFood"]["calories"]                
                del roughdata[y-count]["logId"]
                del roughdata[y-count]["loggedFood"]
                del roughdata[y-count]["favorite"]
                #if the data contains extra nutritionalvalues data slot, delete that also
                if "nutritionalValues" in roughdata[y-count].keys():                
                    del roughdata[y-count]["nutritionalValues"]                
                roughdata[y-count]["calories"] = cal
            else:
                #delete data outside of date range
                del roughdata[y-count]
                count +=1
        
        data = np.append(data, roughdata)
           
#get calories for each day
calories = data[0]["calories"]
totCalories = np.empty((0,1))
dates = np.empty((0,1))


for x in range(1, len(data)):
    
    #if next food entry is from the same date
    if data[x-1]["logDate"] == data[x]["logDate"]:
        
        #add to calorie counter for the day
        calories += data[x]["calories"]
        
        #add to list of calories per day if at end of loop
        if x == len(data)-1:
            totCalories = np.vstack((totCalories, calories))
    #add to list of calories per day if new day
    else:
        dates = np.vstack((dates, data[x-1]["logDate"]))
        totCalories = np.vstack((totCalories, calories))
        calories = data[x]["calories"]
    
totCalories = np.flip(totCalories)
print(totCalories)

[[1052.]
 [ 932.]
 [ 601.]
 [ 810.]
 [ 775.]
 [ 757.]
 [1021.]
 [2191.]
 [ 658.]
 [ 744.]
 [1097.]
 [ 632.]
 [ 857.]
 [1611.]
 [1261.]]


In [4]:
#GET RELATIVE CALORIE DATA

#calculate calories burned more/less than consumed based on weight change
def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*442.39)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*70
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*265
    calories = np.append(calories, [0])

    return calories
        
#calculate total calories burned in a day
calories = caloriesBurned(weight, fat)
print(calories)
for x in range(len(calories)):
    calories[x] = totCalories[x] - calories[x]

#calories = np.append(calories, [2000,2000, 2000])
print(calories)

[-142.30229677 -615.3131631    91.65866162   -0.          488.58337522
   58.17978814 -273.61167113   95.13531859 -121.76220858   -0.
  360.84063081 -348.23303598  367.29365004 -131.07102722    0.        ]
[1194.30229677 1547.3131631   509.34133838  810.          286.41662478
  698.82021186 1294.61167113 2095.86468141  779.76220858  744.
  736.15936919  980.23303598  489.70634996 1742.07102722 1261.        ]


In [5]:
#things to fill in
rootdir = "maddy driscoll/MyFitbitData5/Mad/Physical Activity"
regex = re.compile('heart_rate-2021-03-([1][3-9]|[2][0-7])')
#get heart rates

filenames = os.listdir(rootdir)
heartRates = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            heartRates = np.append(heartRates, file)

#read files for hr data
inputs = np.empty((0,3))
newInput = np.empty(3)
dateTime = []
start = False
minInputs = np.empty((0,3))
hrDates = np.empty((0,1))

date = startDate
inputs = np.empty((0, 3))
tempInputs = np.empty((0, 3))
count = 0


#read heart rate data files
for x in range(np.size(heartRates)):
    filename = rootdir + "/" + heartRates[x]
    data = []
    with open(filename) as json_file:
        data = json.load(json_file)
        tempInputs = np.empty((len(data), 3))
        
        #get inputs from heart rate files
        for y in range(len(data)):  
            #get time of data recording
            temp = data[y]["dateTime"][9:] 
            #convert time input to seconds through the day
            tempInputs[y][0] = int(temp[:2])*3600  + int(temp[3:5])*60 + int(temp[6:9])
            tempInputs[y][1] = int(data[y]["value"]["bpm"])
            #set third input to day of recording
            tempInputs[y][2] = x
        if x == 0:
            inputs = tempInputs
        else:
            #add data recording to list of data records
            inputs = np.vstack((inputs, tempInputs))


for x in range(5):
    condensedInputs =[]
    for y in range(1, len(inputs), 2):    
        if inputs[y][2] == inputs[y-1][2]:
            groupInput = inputs[y] + inputs[y-1]
            groupInput[1] = groupInput[1]/2
            groupInput[2] = groupInput[2]/2
            condensedInputs.append(groupInput)
    inputs = condensedInputs

day = 0
splitInputs = []
lastSplit = 0
for x in range(len(inputs)):
    if inputs[x][2] != day:
        splitInputs.append(x - lastSplit)
        lastSplit = x
        day +=1

splitInputs.append(len(inputs) - lastSplit)
        
        


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
for x in range(len(inputs)):
    if x < len(inputs)-1:
        inputs[x][0] = inputs[x+1][0] - inputs[x][0]
inputs = np.delete(inputs, len(inputs)-1, 0)
splitInputs[len(splitInputs)-1] -=1



#normalize, scale and format

train_data_normalized2 = scaler.fit_transform(np.transpose(inputs)[1] .reshape(-1,1))
scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized1 = scaler.fit_transform(np.transpose(inputs)[0] .reshape(-1,1))

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized3 = scaler.fit_transform(np.transpose(inputs)[2] .reshape(-1,1))

train_data_normalized = np.hstack((train_data_normalized1, train_data_normalized2))

train_data_normalized = np.hstack((train_data_normalized, train_data_normalized3))


inputs = torch.FloatTensor(train_data_normalized)
numOfInputs = np.size(np.transpose(inputs)[0])
print(np.shape(inputs))
print(inputs)

torch.Size([5256, 3])
tensor([[ 0.9531, -0.9108, -1.0000],
        [ 0.9440, -0.8672, -1.0000],
        [ 0.9443, -0.9345, -1.0000],
        ...,
        [ 0.9445, -0.7859,  1.0000],
        [ 0.9440, -0.8068,  1.0000],
        [ 0.9440, -0.8186,  1.0000]])


In [6]:
#calculate expected outputs
y_actual = []
day = 0




scaler = MinMaxScaler(feature_range=(-1,1))
normalized_y = torch.FloatTensor(scaler.fit_transform(calories .reshape(-1,1)))
print(np.shape(normalized_y))

outputs = normalized_y
print(np.shape(outputs))
print(outputs)

torch.Size([15, 1])
torch.Size([15, 1])
tensor([[ 0.0035],
        [ 0.3937],
        [-0.7536],
        [-0.4213],
        [-1.0000],
        [-0.5442],
        [ 0.1144],
        [ 1.0000],
        [-0.4547],
        [-0.4942],
        [-0.5029],
        [-0.2331],
        [-0.7753],
        [ 0.6089],
        [ 0.0772]])


In [7]:
class MyRNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 50, output_size = 1):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden
    
    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [8]:
rnn = MyRNN()
print(rnn)

MyRNN(
  (in2hidden): Linear(in_features=53, out_features=50, bias=True)
  (in2output): Linear(in_features=53, out_features=1, bias=True)
)


In [9]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)   # optimize all cnn parameters
loss_fn = torch.nn.MSELoss(size_average=False)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 30)

C:\Users\peter\.conda\envs\ailab\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:



for epoch in range(500):
    for x in range(12):
        hidden_state = rnn.init_hidden() 
        if x == 0:
            dailyInputs = inputs[:int(splitInputs[x])]            
        else:           
            dailyInputs = inputs[int(sum(splitInputs[:x])) :int(sum(splitInputs[:x])) + int(splitInputs[x])]
        for y in range(int(splitInputs[x])):            
            output, hidden_state = rnn(dailyInputs[y].view(1,3), hidden_state)                               # rnn output
       
        loss = loss_fn(output, outputs[x].view(1, 1))                   #  loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients
    
    print('Epoch: ', epoch, '| train loss: %.4f' % loss.data)



Epoch:  0 | train loss: 0.3529
Epoch:  1 | train loss: 0.2394
Epoch:  2 | train loss: 0.0013
Epoch:  3 | train loss: 0.0242
Epoch:  4 | train loss: 0.0010
Epoch:  5 | train loss: 0.0035
Epoch:  6 | train loss: 0.0036
Epoch:  7 | train loss: 0.0029
Epoch:  8 | train loss: 0.0039
Epoch:  9 | train loss: 0.0032
Epoch:  10 | train loss: 0.0035
Epoch:  11 | train loss: 0.0032
Epoch:  12 | train loss: 0.0032
Epoch:  13 | train loss: 0.0032
Epoch:  14 | train loss: 0.0032
Epoch:  15 | train loss: 0.0032
Epoch:  16 | train loss: 0.0031
Epoch:  17 | train loss: 0.0031
Epoch:  18 | train loss: 0.0031
Epoch:  19 | train loss: 0.0031
Epoch:  20 | train loss: 0.0031
Epoch:  21 | train loss: 0.0031
Epoch:  22 | train loss: 0.0031
Epoch:  23 | train loss: 0.0031
Epoch:  24 | train loss: 0.0031
Epoch:  25 | train loss: 0.0031
Epoch:  26 | train loss: 0.0031
Epoch:  27 | train loss: 0.0031
Epoch:  28 | train loss: 0.0031
Epoch:  29 | train loss: 0.0031
Epoch:  30 | train loss: 0.0031
Epoch:  31 | train

Epoch:  252 | train loss: 0.0127
Epoch:  253 | train loss: 0.0066
Epoch:  254 | train loss: 0.0119
Epoch:  255 | train loss: 0.0104
Epoch:  256 | train loss: 0.0149
Epoch:  257 | train loss: 0.0128
Epoch:  258 | train loss: 0.0155
Epoch:  259 | train loss: 0.0149
Epoch:  260 | train loss: 0.0170
Epoch:  261 | train loss: 0.0198
Epoch:  262 | train loss: 0.0331
Epoch:  263 | train loss: 0.0261
Epoch:  264 | train loss: 0.0378
Epoch:  265 | train loss: 0.0022
Epoch:  266 | train loss: 0.2084
Epoch:  267 | train loss: 0.0966
Epoch:  268 | train loss: 0.0013
Epoch:  269 | train loss: 0.1461
Epoch:  270 | train loss: 0.0042
Epoch:  271 | train loss: 0.1137
Epoch:  272 | train loss: 0.0000
Epoch:  273 | train loss: 0.0705
Epoch:  274 | train loss: 0.0034
Epoch:  275 | train loss: 0.0500
Epoch:  276 | train loss: 0.0056
Epoch:  277 | train loss: 0.0439
Epoch:  278 | train loss: 0.0044
Epoch:  279 | train loss: 0.0661
Epoch:  280 | train loss: 0.0008
Epoch:  281 | train loss: 0.0999
Epoch:  28

In [11]:
for x in range(15):
    hidden_state = rnn.init_hidden()

    if x == 0:
        dailyInputs = inputs[:int(splitInputs[x])]            
    else:           
        dailyInputs = inputs[int(sum(splitInputs[:x])) :int(sum(splitInputs[:x])) + int(splitInputs[x])]
    
    for y in range(int(splitInputs[x])):            
        output, hidden_state = rnn(dailyInputs[y].view(1,3), hidden_state)
    print(output[0][0], " : ", outputs[x][0], 'train loss: %.4f' % loss.data)
    loss = loss_fn(output, outputs[x].view(1, 1))  
    print('train loss: %.4f' % loss.data)
      

tensor(0.3252, grad_fn=<SelectBackward0>)  :  tensor(0.0035) train loss: 0.3544
train loss: 0.1035
tensor(0.1365, grad_fn=<SelectBackward0>)  :  tensor(0.3937) train loss: 0.1035
train loss: 0.0661
tensor(-0.3840, grad_fn=<SelectBackward0>)  :  tensor(-0.7536) train loss: 0.0661
train loss: 0.1366
tensor(0.0406, grad_fn=<SelectBackward0>)  :  tensor(-0.4213) train loss: 0.1366
train loss: 0.2134
tensor(-0.7930, grad_fn=<SelectBackward0>)  :  tensor(-1.) train loss: 0.2134
train loss: 0.0428
tensor(-0.1577, grad_fn=<SelectBackward0>)  :  tensor(-0.5442) train loss: 0.0428
train loss: 0.1494
tensor(0.1696, grad_fn=<SelectBackward0>)  :  tensor(0.1144) train loss: 0.1494
train loss: 0.0031
tensor(0.3043, grad_fn=<SelectBackward0>)  :  tensor(1.) train loss: 0.0031
train loss: 0.4840
tensor(-0.3531, grad_fn=<SelectBackward0>)  :  tensor(-0.4547) train loss: 0.4840
train loss: 0.0103
tensor(-0.3986, grad_fn=<SelectBackward0>)  :  tensor(-0.4942) train loss: 0.0103
train loss: 0.0091
tensor(